In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from functions import get_bet_ev, get_bet_return, get_ev_from_df, custom_cv_eval

In [2]:
%run MODEL_LIST.ipynb            #Load the models

In [3]:
df = pd.read_csv("../data/ufc-master.csv")

In [4]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [5]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [6]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [7]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4011
4011
4011


In [8]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
201,Volkan Oezdemir,Ilir Latifi,-190.0,165.0,52.631579,165.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,2.0,-10.16,-5.08,-7.0,-2.590000,0.500000,1.460000,0.0,1.0,0
202,Rodolfo Vieira,Oskar Piechota,-225.0,185.0,44.444444,185.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,0.0,0.00,7.62,0.0,1.280000,0.300000,-4.010000,0.0,1.0,0
203,Enrique Barzola,Bobby Moffett,-165.0,145.0,60.606061,145.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,1.0,7.62,7.62,1.0,-0.520000,1.200000,-1.820000,0.0,1.0,0
204,Gilbert Burns,Aleksei Kunchenko,120.0,-140.0,120.000000,71.428571,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,-4.0,-5.08,-2.54,-2.0,0.530000,-0.700000,-2.100000,0.0,1.0,0
205,Ciryl Gane,Raphael Pessoa Nunes,-470.0,375.0,21.276596,375.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,0.0,-2.54,-7.62,-1.0,-2.720000,-0.900000,-1.320000,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,Duane Ludwig,Darren Elkins,-155.0,135.0,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.00,2.54,6.0,-13.666667,0.000000,0.000000,0.0,1.0,1
4280,John Howard,Daniel Roberts,-210.0,175.0,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,7.62,7.62,-2.0,-18.000000,-1.000000,-4.666667,0.0,1.0,0
4281,Brendan Schaub,Chase Gormley,-260.0,220.0,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,-2.54,-2.12,0.0,-4.000000,1.000000,1.000000,0.0,1.0,0
4282,Mike Pierce,Julio Paulino,-420.0,335.0,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,10.16,7.62,-5.0,-40.500000,0.000000,-3.500000,0.0,1.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
0,Tyron Woodley,Gilbert Burns,-170.0,150.0,58.823529,150.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,2.54,-7.62,-5.0,0.63,0.2,1.18,1.0,1.0,1
1,Blagoy Ivanov,Augusto Sakai,-105.0,-115.0,95.238095,86.956522,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,10.16,10.16,-4.0,2.42,-0.2,-0.83,1.0,1.0,1
2,Billy Quarantillo,Spike Carlyle,-130.0,110.0,76.923077,110.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-5.08,2.54,-4.0,3.35,-4.0,-0.80,1.0,1.0,0
3,Roosevelt Roberts,Brok Weaver,-360.0,300.0,27.777778,300.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-5.08,0.00,2.0,2.70,0.3,-1.84,1.0,1.0,0
4,Mackenzie Dern,Hannah Cifers,-420.0,335.0,23.809524,335.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-7.62,-2.54,0.0,1.58,-0.6,-0.14,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Andre Ewell,Jonathan Martinez,-133.0,113.0,75.187970,113.000000,2020-02-08,"Houston, Texas, USA",USA,Red,...,0.0,0.00,-12.70,7.0,0.10,0.0,0.42,0.0,1.0,0
197,Austin Lingo,Youssef Zalel,-210.0,175.0,47.619048,175.000000,2020-02-08,"Houston, Texas, USA",USA,Blue,...,0.0,0.00,5.08,2.0,0.00,0.0,0.00,0.0,1.0,1
198,Valentina Shevchenko,Liz Carmouche,-1100.0,700.0,9.090909,700.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,-1.0,2.54,0.00,-4.0,-0.24,-0.1,0.63,0.0,1.0,0
199,Vincente Luque,Mike Perry,-250.0,210.0,40.000000,210.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,-3.0,-2.54,-10.16,0.0,-0.91,-1.0,-0.21,0.0,1.0,0


In [29]:
def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    #If the list is empty we can just return it without doing anything
    number_of_features = len(cur_features)
    df_sel = df[cur_features]
    df_sel = df_sel.dropna()
    df_sel = pd.get_dummies(df_sel)
    labels_sel = labels[labels.index.isin(df_sel.index)]
    odds_sel = odds[odds.index.isin(df_sel.index)]        
    orig_score = custom_cv_eval(df_sel, m, labels_sel, odds_sel, get_total=True, min_ev = min_ev)
    #print(orig_score)
    best_features = cur_features
    best_score = orig_score
    print(f"The original score is {orig_score}")
    if number_of_features == 0:
        return []
    
    for z in range(number_of_features):
        temp_features = cur_features.copy()
        #Remove a feature
        del temp_features[z]
        df_sel = df[temp_features]
        df_sel = df_sel.dropna()
        df_sel = pd.get_dummies(df_sel)
        labels_sel = labels[labels.index.isin(df_sel.index)]
        odds_sel = odds[odds.index.isin(df_sel.index)]        
        temp_score = custom_cv_eval(df_sel, m, labels_sel, odds_sel, get_total=True, min_ev = min_ev)
        if temp_score > best_score:
            best_features = temp_features
            best_score = temp_score
            print(f"NEW BEST FEATURE SET")
            print(best_features)
            print(best_score)
        else:
            print("Score: ", temp_score)
        
        #Get a score
    if best_features != cur_features:
        return remove_to_improve(best_features, m, df, labels, odds, scale, min_ev)
    else:
        return best_features

In [39]:
temp_features = remove_to_improve(features_5, model_5, df_train, label_train, odds_train, min_ev = min_ev_5)

The original score is 8050.180486406278
NEW BEST FEATURE SET
['avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds', 'R_Stance']
9278.454792418263
Score:  8371.644712306645
Score:  7866.755216046422
Score:  4097.943949426179
Score:  7700.650422076472
Score:  7002.255464492751
NEW BEST FEATURE SET
['R_win_by_Decision_Unanimous', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
14128.796888069486
The original score is 14128.796888069486
NEW BEST FEATURE SET
['avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
14492.350116222467
Score:  10141.170442735665
Score:  5566.563285168149
Score:  3939.509241944281
Score:  11513.874254677383
Score:  -9166.057759887555
The original score is 14492.350116222467
Score:  9529.76974764657
Score:  5798.938666469415
Score:  4242.869658962085
Score:  12545.450250672517
Score:  -10687.96529413737


In [40]:
print(temp_features)

['avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
